# Simple structured data demo

This is a very basic demo of how to log structured datasets for classification tasks with XGBoost.

In [ ]:
import os

os.environ['GALILEO_CONSOLE_URL']="http://localhost:8088"
os.environ["GALILEO_USERNAME"]="user@example.com"
os.environ["GALILEO_PASSWORD"]="Th3secret_"

import dataquality as dq
dq.configure()

dq.init("structured_classification", "my-proj", "my-run")

## 1. Load data

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.datasets import load_wine

wine = load_wine()

X = wine.data
y = wine.target

# When exporting to hdf5 you can't have col names containing forward slash
feature_names = [ft_name.replace("/", "|") for ft_name in wine.feature_names]


def understand_dataset(dataset) -> None:
    for elem in dir(dataset):
        print(elem, type(getattr(dataset, elem)))

    print(dataset.shape, y.shape)
    
#understand_dataset(wine)

## 2. Create and fit model on training data

In [ ]:
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(X, y)

## 3. Set labels for run

In [ ]:
target_names = wine.target_names
dq.set_labels_for_run(target_names)

## 4. Log data!

In [ ]:
dq.log(
    model=fitted_model,
    X=X,
    y=y,
    feature_names=feature_names,
    split="training"
)

In [ ]:
dq.log(
    model=fitted_model,
    X=X,
    y=y,
    feature_names=feature_names,
    split="test"
)

In [ ]:
dq.log(
    model=fitted_model,
    X=X,
    feature_names=feature_names,
    split="inference",
    inference_name="my-inf-name"
)

## 5. Call finish to start processing

In [ ]:
dq.finish()